In [196]:
import numpy as np
import matplotlib.pyplot as plt
from astroquery.vizier import Vizier
import HERA_hack

In [2]:
Vizier.ROW_LIMIT = -1
catalog_list = Vizier.find_catalogs('GLEAM')
catalogs = Vizier.get_catalogs(catalog_list.keys())


In [3]:
'''
--------------------------------------------------------------------------------
 FileName        Lrecl  Records   Explanations
--------------------------------------------------------------------------------
ReadMe              80        .   This file
table1.dat          39       28   GLEAM first year observing parameters
gleamegc.dat      3155   307455   GLEAM EGC catalog, version 2
GLEAM_EGC_v2.fits 2880   137887   FITS version of the catalog
--------------------------------------------------------------------------------
'''

#We will extract version 2 catalogue
tt = catalogs['VIII/100/gleamegc'] 

#List all the keys 
#Details of the keys are available here: http://cdsarc.u-strasbg.fr/ftp/cats/VIII/100/ReadMe
#And in more details here: https://heasarc.gsfc.nasa.gov/W3Browse/all/gleamegcat.html

print(tt.keys())

src_name = tt['GLEAM'] #Source ID
RA       = tt['RAJ2000'] #RA
DEC      = tt['DEJ2000'] #DEC
flux     = tt['Fpwide'] #Peak flux in wide (170-231MHz) image

['GLEAM', 'RAJ2000', 'DEJ2000', 'Fpwide', 'Fintwide', 'eabsFpct', 'efitFpct', 'Fp076', 'Fint076', 'Fp084', 'Fint084', 'Fp092', 'Fint092', 'Fp099', 'Fint099', 'Fp107', 'Fint107', 'Fp115', 'Fint115', 'Fp122', 'Fint122', 'Fp130', 'Fint130', 'Fp143', 'Fint143', 'Fp151', 'Fint151', 'Fp158', 'Fint158', 'Fp166', 'Fint166', 'Fp174', 'Fint174', 'Fp181', 'Fint181', 'Fp189', 'Fint189', 'Fp197', 'Fint197', 'Fp204', 'Fint204', 'Fp212', 'Fint212', 'Fp220', 'Fint220', 'Fp227', 'Fint227', 'alpha', 'Fintfit200']


In [68]:
################## Set up obs #######################
dishes = np.array([[0,0],[0,55],[30,30],[0,60],[2,55],[47,2],[45,23],[56,21],[30,115],[48,52],[100,100],[0,200],[115,30],[33,31],[49,11],[21,24],[25,6],[56,9],[12,13],[16,17],[38,17],[60,14],[26,28],[6,45],[3,37],[12,55],[200,0],[145,13],[134,65],[139,163]])

#observable corners of the sky [lat,long]
acorner = np.array([[120,270],[122,280],[120,280],[122,270]])

HERA = HERA_hack.telescope(dishes, latitude=-30, channel_width=1., Tsys=300, beam_width=3, beam = 'gaussian')

obs = HERA_hack.observation(HERA, 100, 100, 0.01,acorner,1, 0.2, norm = False, pbeam = True)

obs.compute_beam()

###################################################


In [168]:
#Convert catalog flux unit to mJy 
flux_mjy = flux*1000


#convert to equatorial coords (only DEC is changed)
DEC_co_lat = 90-DEC 
#convert to radians 
RA_rads = RA*(np.pi/180.)
DEC_rads = DEC_co_lat*(np.pi/180.)


psource_flux = []
psource_RA = []
psource_DEC = []

#INITIAL CUT: Pick out only the sources bright enough to be seen at center of beam
for i in range(len(flux)):
    if flux[i] >= 0.100: 
        psource_flux.append(flux[i])
        psource_RA.append(RA_rads[i])
        psource_DEC.append(DEC_rads[i])
        
#convert to array for ease of use
psource_flux = np.asarray(psource_flux)
psource_RA = np.asarray(psource_RA)
psource_DEC = np.asarray(psource_DEC)



In [169]:
len(psource_flux)

179248

In [170]:
co_lat = np.pi / 2. - obs.latitude

#DEC distance to centre of the beam, may need to do actual distance...
dist_from_centre = np.abs(psource_DEC-co_lat)

data = np.stack([psource_flux,psource_RA,psource_DEC,dist_from_centre], axis = 1)

psource_data = data[np.argsort(data[:, 3])] #sort by distance from centre of beam 
#(becuase that way you do the brightest possiblesources first = less computing time)

psource_data

array([[1.25535100e+00, 5.29811903e-01, 2.09439573e+00, 6.28318531e-07],
       [3.47007000e-01, 4.11382329e+00, 2.09439620e+00, 1.09955743e-06],
       [5.38232000e-01, 4.31637320e+00, 2.09439723e+00, 2.12930169e-06],
       ...,
       [2.40826000e-01, 2.63321491e+00, 1.04727063e+00, 1.04712447e+00],
       [3.75251000e-01, 7.30076916e-01, 1.04724000e+00, 1.04715510e+00],
       [1.53287000e-01, 8.33345709e-01, 1.04721520e+00, 1.04717991e+00]])

In [90]:

#Building Pbeam
phis = (2. * np.pi * obs.times) + obs.position[0,1]

primary = np.zeros((obs.Nt, psource_data.shape[0]))

for i in range(obs.Nt): #compute the elements of pbeam
    primary[i] = np.exp(-((psource_data[:,1]-phis[i])**2 +(psource_data[:,2]-co_lat)**2) / float(obs.beam_width**2))# 2D gaussian beam (N_position,2) 
    #this primary beam should now be in order of closest to furthest 

[0.5298119  4.11382329 4.3163732  ... 2.63321491 0.73007692 0.83334571]
4.71238898038469
[0.5298119  4.11382329 4.3163732  ... 2.63321491 0.73007692 0.83334571]
4.775220833456485
[0.5298119  4.11382329 4.3163732  ... 2.63321491 0.73007692 0.83334571]
4.838052686528282


In [186]:
#find the maximum primary per bin for all times
#if it is bring enough in one time that is enought to need it in sim.

#1: bin pbeams and psource_data ( not with hist, juts with index numbers)

nbins = 48
bin_index = np.int(len(primary[1])/nbins)

psource_final = []
for i in range(nbins):
    #find max pbeam of all time
    lower_bound = i*bin_index
    upper_bound = (i+1)*bin_index
    maxes = []
    for j in range(obs.Nt): 
        maxes.append(max(primary[j,lower_bound:upper_bound]))
    #print('done time %s' %i)
    maxi = max(maxes) #This is now the max pbeam you use to check the fluxes
    
    for k in range(bin_index):
        #print('done source %s'%k)
        if psource_data[((i+1)*k),0]* maxi >= .100: #find the bright guys
            #print(psource_data[((i+1)*k),:3],  psource_data[((i+1)*k),0]* maxi )
            psource_final.append(psource_data[((i+1)*k),:3]) #append bright guys to final list
        else:
            continue
psource_final = np.asarray(psource_final)

z_coord = np.zeros((psource_final.shape[0],1))

psource_final = np.concatenate((psource_final, z_coord), axis = 1)



In [187]:
#Convert to 3D coords        
def convert_to_3d(psource_final): 
for i in range(psource_final.shape[0]):                       
    psource_final[i,1] = np.sin(psource_final[i,1])*np.cos(psource_final[i,2])#X
    psource_final[i,2] = np.sin(psource_final[i,1])*np.sin(psource_final[i,2])#Y
    psource_final[i,3] = np.cos(psource_final[i,1])#Z                       
return psource_final


## return psource_flux, psource_transformed

# Then the psource_fluxes get converted into Tb and concatenated to the sky array AT THE VERY VERY END
# the tranformed coordinates get concatenated to r in bdotr
# convert to T_b = 1.222 *10**3*((psourceflux/1000)/(nu**2*theta_res * phi_res))

# index = np.where(primary == max(primary[1]))


In [188]:
convert_to_3d(psource_final)

array([[ 1.25535100e+00, -2.52685796e-01, -2.16510918e-01,
         9.68244451e-01],
       [ 3.47007000e-01,  4.13073105e-01,  3.47644763e-01,
         9.15891526e-01],
       [ 5.38232000e-01,  4.61304305e-01,  3.85481683e-01,
         8.95472692e-01],
       ...,
       [ 1.95758250e+01, -4.93456271e-02, -4.43197746e-02,
         9.98782752e-01],
       [ 1.56745700e+01,  7.41291280e-03,  6.09007961e-03,
         9.99972524e-01],
       [ 2.22658670e+01,  4.44976248e-01,  3.53018754e-01,
         9.02620889e-01]])

In [189]:
psource_final[:,1:]

array([[-0.2526858 , -0.21651092,  0.96824445],
       [ 0.4130731 ,  0.34764476,  0.91589153],
       [ 0.4613043 ,  0.38548168,  0.89547269],
       ...,
       [-0.04934563, -0.04431977,  0.99878275],
       [ 0.00741291,  0.00609008,  0.99997252],
       [ 0.44497625,  0.35301875,  0.90262089]])

In [193]:
pos = np.zeros((3,3))

In [195]:
np.concatenate((pos,psource_final[:,1:]), axis = 0)

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       ...,
       [-0.04934563, -0.04431977,  0.99878275],
       [ 0.00741291,  0.00609008,  0.99997252],
       [ 0.44497625,  0.35301875,  0.90262089]])